In [1]:
import torch
from transformers import RobertaConfig, RobertaModel, AutoTokenizer, RobertaTokenizerFast
import pandas

/home/david/anaconda3/envs/GenoPhen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file = r"/media/david/T7/NCBI/E.Coli.Shigella/PDG000000004.3840.amr.metadata.tsv"
raw_data = pandas.read_csv(file, sep='\t')

/tmp/ipykernel_24605/1932442386.py:2: DtypeWarning: Columns (1,5,6,30,55) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pandas.read_csv(file, sep='\t')


In [3]:
genotype_data = raw_data[~raw_data.AMR_genotypes_core.isnull()]
phenotype_data = raw_data[~raw_data.AST_phenotypes.isnull()]

In [4]:
gene_vocab = genotype_data.AMR_genotypes_core.values.tolist()
gene_vocab = [g.split(',') for g in gene_vocab]
gene_vocab = [item for sublist in gene_vocab for item in sublist]
gene_vocab = list(set(gene_vocab))
with open('gene_vocabulary.txt', 'w') as f:
    for line in gene_vocab:
        f.write(f"{line}\n")
len(gene_vocab)

1398

In [5]:
def mask_three_geno_fn(x, min_len=3):
    all_items = x['AMR_genotypes_core'].split(',')
    all_items = [a for a in all_items if "=PARTIAL" not in a]
    all_items = [a for a in all_items if "=PARTIAL_END_OF_CONTIG" not in a]
    return len(all_items) >= min_len

In [24]:
three_amr = genotype_data[genotype_data.apply(mask_three_geno_fn, axis=1)]
genotype_corpus = genotype_data.AMR_genotypes_core.values.tolist()
#genotype_corpus = [[gg.split('_')[0] for gg in g.split(',')] for g in genotype_corpus] #Removes detail from point mutation and only keeps the gene
#genotype_corpus = [" ".join(g) for g in genotype_corpus]
genotype_corpus = [g.replace(',', ' ') for g in genotype_corpus]
example = genotype_corpus[1337]
example

'glpT_E448K=POINT marR_S3N=POINT pmrB_E123D=POINT'

In [25]:
def get_genotype_corpus():
    for start_idx in range(0, len(genotype_corpus), 1000):
        samples = genotype_corpus[start_idx: start_idx + 1000]
        yield samples

In [35]:
training_genotype_corpus = get_genotype_corpus()
old_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", tokenizer_type='bert', clean_text=False)
tokenizer = old_tokenizer.train_new_from_iterator(training_genotype_corpus, 52000)
tokens = tokenizer.tokenize(example)
tokens

['glpt',
 '_',
 'e448k',
 '=',
 'point',
 'marr',
 '_',
 's3n',
 '=',
 'point',
 'pmrb',
 '_',
 'e123d',
 '=',
 'point']

In [22]:
example

'glpT_E448K=POINT marR_S3N=POINT pmrB_E123D=POINT'

In [10]:
example.split(' ')[0].split('_')[0]

'glpT'

In [11]:
genotype_corpus[123].split(' ')[0].split('_')[0]

'aac(3)-VIa'